# Process from webcam

In [ ]:
import cv2
from deepface import DeepFace
import FaceDlibs
import numpy as np
import math

# Captura de video desde la cámara
vid = cv2.VideoCapture(0)

# Interfaz de detectores de cara
FDet = FaceDlibs.FaceDlib()

# Parámetros de texto
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0) 
thickness = 2 
pos = (50, 50)

# Cargamos la imagen con el canal alfa
filter_image = cv2.imread('emojis/gota.png', -1)
star_img = cv2.imread('emojis/happy.png', -1)
barba_img = cv2.imread('emojis/barba.png', -1)


while(True):      
    ret, frame = vid.read()
    B, G, R = cv2.split(frame)

    if ret:
        # Analizamos la imagen para obtener los atributos de la cara con Deepface
        obj = DeepFace.analyze(img_path = frame, enforce_detection=False, actions=['gender', 'emotion'])
        print(obj)
        if len(obj) == 0:
            break
        
        # Carácteristicas y valores de la cara capturada
        face = obj[0]
        
        if face['dominant_emotion'] == "sad":
            # Detectamos los ojos
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[0])

            if values is not None:
                # Valores de la posición de la cara, ojos y forma
                face, eyes, shape = values
                # Coordenadas y dimensión de la ROI de la cara
                [x, y, w, h] = face
                if x > -1:
                    # Coordenadas y dimensión de los ojos (lex: left eye X, ley: left eye Y, rex: right eye X, rey: right eye Y)
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        # Calcular el ángulo de rotación entre los ojos izquierdo y derecho
                        dx = rex - lex
                        dy = rey - ley
                        angle = math.atan2(dy, dx)  # Ángulo en radianes
                        angle_degrees = math.degrees(angle)  # Convertir a grados

                        # Redimensionamos el filtro
                        filter_resized = cv2.resize(filter_image, (40, 40))

                        # Crear una matriz de rotación para ajustarse a la rotación
                        center = (filter_resized.shape[1] // 2, filter_resized.shape[0] // 2)
                        rotation_matrix = cv2.getRotationMatrix2D(center, angle_degrees, 1)
                        rotated_filter = cv2.warpAffine(filter_resized, rotation_matrix, 
                                                        (filter_resized.shape[1], filter_resized.shape[0]))

                        
                        # Colocar el filtro rotado sobre el ojo izquierdo (punto 39 de los puntos carácteristicos de la cara de DLIB)
                        x, y = shape[39]
                        x = x - 40
                        y = y + 10
                        for i in range(rotated_filter.shape[0]):
                            for j in range(rotated_filter.shape[1]):
                                if rotated_filter[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y + i, x + j] = rotated_filter[i, j][0:3]

                        # Posicionar el filtro sobre el ojo derecho (punto 42)
                        x, y = shape[42]
                        x = x + 10
                        y = y + 10
                        for i in range(rotated_filter.shape[0]):
                            for j in range(rotated_filter.shape[1]):
                                if rotated_filter[i, j][3] > 0:  # Si la transparencia es mayor que 0
                                    frame[y + i, x + j] = rotated_filter[i, j][0:3]

        elif face['dominant_emotion'] == "happy":
            # Obtener las coordenadas de los ojos
            if face['region']['left_eye'] != None or face['region']['right_eye'] != None:

                left_eye = face['region']['left_eye']
                right_eye = face['region']['right_eye']


                # Redimensionar la imagen del filtro a las dimensiones de los ojos detectados
                star_resize = cv2.resize(star_img, (50, 50))

                # Colocar el filtro sobre el ojo izquierdo (sin considerar transparencia)
                for i in range(star_resize.shape[0]):
                    for j in range(star_resize.shape[1]):
                        if star_resize[i, j][3] > 0:  # Si la transparencia es mayor que 0
                            frame[left_eye[1] + i -25, left_eye[0] + j -25] = star_resize[i, j][0:3]

                # Colocar el filtro sobre el ojo derecho (sin considerar transparencia)
                for i in range(star_resize.shape[0]):
                    for j in range(star_resize.shape[1]):
                        if star_resize[i, j][3] > 0:  # Si la transparencia es mayor que 0
                            frame[right_eye[1] + i -25, right_eye[0] + j-25] = star_resize[i, j][0:3]

        elif face['dominant_emotion'] == "neutral":
            
            # Detectamos la cara, ojos y los puntos carácterísticos de la cara (shape)
            values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[0], FDet.EyeDetectors[0])

            if values is not None:
                face, eyes, shape = values
                # Coordenadas y dimensiones de la ROI de la cara
                [x, y, w, h] = face
                if x > -1:
                    # Media de las coordenadas de los ojos
                    [lex, ley, rex, rey] = eyes
                    if lex > -1:
                        # Redimensionamos el filtro
                        filter_resized = cv2.resize(barba_img, (480, 480))

                        # El pto. 19 es el extremo izq. de la ceja dcha. desde la perspectiva del observador
                        x_i, y_i = shape[19]
                        # El pto. 24 es el extremo dcho. de la ceja izq. desde la perspectiva del observador
                        x_d, y_d = shape[24]
                        # El pto. 27 es el punto superior del puente de la nariz, entre ambas cejas (referencia para centrar el filtro)
                        x_n, y_n = shape[27]
                        # Como la máscara no es perfectamente simétrica, ajustamos con los puntos de las cejas
                        x_m=x_n
                        y_m=y_i
                        for i in range(filter_resized.shape[0]):
                            for j in range(filter_resized.shape[1]):
                                # Si la transparencia es mayor que 0
                                if filter_resized[i, j][3] > 0:  
                                    frame[y_m + i - 170, x_m + j - 220] = filter_resized[i, j][0:3]                    

        # Mostrar el texto con la información de la cara
        image = cv2.putText(frame, str(obj[0]['dominant_gender'] + ' ' + obj[0]['dominant_emotion']), pos, font,
                            fontScale, color, thickness, cv2.LINE_AA) 
        # Mostrar fotograma
        cv2.imshow('Vid', frame)

    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 13.59it/s]


[{'gender': {'Woman': 7.211755216121674, 'Man': 92.78824925422668}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 0.044616995477571815, 'disgust': 0.027485114879950277, 'fear': 0.029948518533916387, 'happy': 86.47115745968566, 'sad': 9.299337673147779, 'surprise': 0.0008377870971391668, 'neutral': 4.126613833106389}, 'dominant_emotion': 'happy'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 11.90it/s]


[{'gender': {'Woman': 17.990601062774658, 'Man': 82.00939297676086}, 'dominant_gender': 'Man', 'region': {'x': 320, 'y': 223, 'w': 72, 'h': 72, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0.94, 'emotion': {'angry': 0.22115944884717464, 'disgust': 0.0005082099960418418, 'fear': 6.343797594308853, 'happy': 90.90811610221863, 'sad': 0.0895241042599082, 'surprise': 0.018966634524986148, 'neutral': 2.417930029332638}, 'dominant_emotion': 'happy'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 13.37it/s]


...

[{'gender': {'Woman': 8.276749402284622, 'Man': 91.72325134277344}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 0.016574377774503248, 'disgust': 0.002222435019256865, 'fear': 63.21868081204181, 'happy': 0.28497226837857176, 'sad': 18.37403287732797, 'surprise': 0.023228258871574484, 'neutral': 18.080289249437406}, 'dominant_emotion': 'fear'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 13.31it/s]


[{'gender': {'Woman': 0.06257956847548485, 'Man': 99.93742108345032}, 'dominant_gender': 'Man', 'region': {'x': 156, 'y': 80, 'w': 206, 'h': 206, 'left_eye': (289, 156), 'right_eye': (224, 160)}, 'face_confidence': 0.94, 'emotion': {'angry': 4.538973793387413, 'disgust': 0.00705731799826026, 'fear': 53.41866612434387, 'happy': 0.6744307465851307, 'sad': 16.07656627893448, 'surprise': 0.03195276949554682, 'neutral': 25.252357125282288}, 'dominant_emotion': 'fear'}]


Action: emotion: 100%|██████████| 2/2 [00:00<00:00, 14.14it/s]


[{'gender': {'Woman': 8.417785167694092, 'Man': 91.58220887184143}, 'dominant_gender': 'Man', 'region': {'x': 0, 'y': 0, 'w': 639, 'h': 479, 'left_eye': None, 'right_eye': None}, 'face_confidence': 0, 'emotion': {'angry': 0.00040419185736187944, 'disgust': 1.618973846277968e-05, 'fear': 65.41124994620681, 'happy': 0.7672457972471955, 'sad': 19.270101872670427, 'surprise': 0.07281037110860693, 'neutral': 14.478164675454536}, 'dominant_emotion': 'fear'}]
